In [1]:
import json

import pandas as pd
import eth_event

# Load Data

In [2]:
transactions = pd.read_csv('data/transactions.csv')
transactions.head()

,transaction_hash,block_timestamp,block_number,block_hash,from_address,to_address,value,input
0,0xd6125f1b07b1a3004b96699d4eaff2874b5b1038bd15...,2022-12-09 00:04:23 UTC,16143452,0x5f109b09a1dfa4db9ec8a8d81f2d3949393c64b7b254...,0x368f27605a0dc5a8cfd04eec4c6ed80a29697e61,0x3421ca98dc9689e82da4e5cb23f6ad47f8707aa1,62175500000000000,0x
1,0xb6dc2685b77c383590a88ab805a10ad6e27c8a4bbc90...,2022-12-09 00:08:11 UTC,16143471,0xe9993e1c766ee7811889c2b250b50d60a5e14c42769c...,0x71660c4005ba85c37ccec55d0c4493e66fe775d3,0x33fe0557cfe59afdc57b45062ec61acf4f2f3971,1159095000000000,0x
2,0xa63a0e949d234cd88b3ac65e983bfa989f2ac23f4458...,2022-12-09 00:09:35 UTC,16143478,0x2bda3ae6e1336f5b4431dc2f788e7cf1bb91d7ad0d9f...,0x562fead53d265fc0b28bf6a3ec39bc4b50b5a94d,0x0a5df7e60428fca0fea41702edf54408f5161294,156141000000000000,0x
3,0x68bf7ba653a253c5f7046ac9e2b1043e9ee7095a2495...,2022-12-08 23:42:11 UTC,16143343,0x35994d2eb5e48ca8c46ef330e8112772448e957d3587...,0xb97bf5f5afc88e34393b6bef38da605999f42f82,0xf89d7b9c864f589bbf53a82105107622b35eaa40,4599618625205563000,0x
4,0x518597f78434d5787d9903da41e7ee1861bfa96925b5...,2022-12-08 23:42:11 UTC,16143343,0x35994d2eb5e48ca8c46ef330e8112772448e957d3587...,0xddfabcdc4d8ffc6d5beaf154f18b778f892a0740,0xf6281d13c616900767d70cafdd6b2bd63dd948a5,321000000000000000,0x


In [3]:
logs = pd.read_json('data/logs.jsonlines', orient='records', lines=True)
logs.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash
0,137,0x3badabb5347f609b62e045bee7f1e5bc3c51282d1ace...,68,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-08 23:34:47 UTC,16143306,0x90f199a40cca479bd70efa85384bf059bd511a83c2fb...
1,407,0xe8629fb0a60bc00f6e4773ca90073b63901375cd383e...,143,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:13:23 UTC,16143497,0xd6efa95fcd2ff89afc39bdecb652bec07aa42f644987...
2,133,0xed4f486554751d3b27657a424bfb65541d4c579e21ce...,45,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:07:23 UTC,16143467,0x01981d3d143b7c75dc44a214e7164147bf3b5a0542a7...
3,171,0x851252c5ab0e0a752f0fbe8d99ed0369894b09b3c53c...,75,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:07:47 UTC,16143469,0xd75756e7c41d35890b5b111afec801bd9af69e4a727a...
4,271,0xf2cb8d3cc925a0fa9e62c2206b031518c59296a80c06...,89,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:07:59 UTC,16143470,0x93bf128edac58b36192a2223e8120960a74b25eed101...


# Decoding

## ETH Transfers
Good example address: `0x0022ae67605e018894cd7ce2893792ccad9c1a00`

Has three transactions, and all three are simple ETH transfers.

In [4]:
eth = transactions.query('input == "0x"')
eth.shape

(267536, 8)

In [5]:
eth[['transaction_hash', 'block_timestamp', 'from_address', 'to_address', 'value']].sample(20)

,transaction_hash,block_timestamp,from_address,to_address,value
58755,0x8a1d92028046fcdcfa17dd37c37a4433167390d65a64...,2022-12-08 15:59:35 UTC,0x97972fa6d980aa9b93d1b584541055840302de05,0xebbb3ac57d9aefd42542aa1fba02996c286a7aed,3410761568991733
160771,0x194f84c6b4506d4b263a329f93d7ca84609a7d6bc266...,2022-12-08 21:31:35 UTC,0x56eddb7aa87536c09ccc2793473599fd21a8b17f,0xc571736365d4746243b0e1ef99952f6d90cfd6aa,246865040000000000
21235,0x6633df0fb822c54e1f02c20fad90f09c58e43464973b...,2022-12-08 23:31:59 UTC,0x974caa59e49682cda0ad2bbe82983419a2ecc400,0x2b96328454b17a8bf49397573237906c723ba78e,28845200000000000
281597,0x430ec7b39c5ff13a638a5d22ca8f4958cac5f7639c49...,2022-12-08 07:09:59 UTC,0xbd13d0ec59cd97599384f86b524834cf0255184b,0x44abcd183e7caaede253be31e11ef280c8e89127,244906314000000000
110790,0xbc2ec0e65aa5f6e1f4cc90010def7ec630f5181b4ddd...,2022-12-08 08:28:35 UTC,0x94159496b0a642bff87ec373a1a36ba78cfc041b,0xc731cf6f2c5f14779bf089644f8a7f32a07487a9,110000000000000000
74564,0x863c5f6ccbaf818cfc3bb1e215e563229cd1dc9942ab...,2022-12-08 12:55:11 UTC,0x4608bc966a298cbc54cbd7698330f117c3d84a46,0x9dd7b56ec1e5821e6eacb917502ec39b3d1f3dd5,7100000000000000000
229514,0x72842d632effff8cf1803f0c9ce7d83d18f63b8a8027...,2022-12-08 02:26:59 UTC,0x21b9e23a5041f66acfb872357e255a86b529c4d4,0xc000453abf2e736832ab4c228596843908742c88,2017221085272371
18006,0xf7179f508b868db0be9b4a6b76e9390dafc8644b1b38...,2022-12-08 23:06:23 UTC,0xf96ee7b48f729baf1976307dcfc8d1f89f664b1a,0x252bb2809607be747e500ea1d78670ce7126bb0f,161000000000000000
288317,0xfe1fff3853c15f407dedc72bfab215331a566a5ff3f5...,2022-12-08 05:37:23 UTC,0xb287eac48ab21c5fb1d3723830d60b4c797555b0,0xc181c1a6883e50a494f0f55483c98379d07991bf,38400000000000000
17466,0xb23914534b41b3e58da15ea978c6b210a98c33943f40...,2022-12-08 23:02:35 UTC,0x8216874887415e2650d12d53ff53516f04a74fd7,0x8f3acbd4b694acaad2da998952c37ff8b538c947,18900000000000000


In [6]:
occurances = eth.to_address.value_counts() + eth.from_address.value_counts()
occurances[occurances == 3].head(20)

0x0000006094141cd5d88b13aa8845dc34e9b1c3a0    3.0
0x0000007c121b92463d2a67d3194a6927524222a6    3.0
0x0016a464920f36b69d1f7f60f392b2e93eea953f    3.0
0x001708c89f19d2d4f40b0ad1726f38e9f4f4c0f8    3.0
0x0022067645e88fa146f641d3644ce8a7066156ff    3.0
0x0022ae67605e018894cd7ce2893792ccad9c1a00    3.0
0x0029dd4662551c9939e3ad378417100a3fed3b8c    3.0
0x00375c6ff668f7709e066da645d45b2df12482f2    3.0
0x004558ce5b0f9ec2006724c471cd850bd151bbff    3.0
0x004800859504d1f16ad8447c97c4a2223d2e395e    3.0
0x004f39ef539f4e9fd2d4d2523b529a4711bbd539    3.0
0x005171631fcf544daf2682d0fc97c83dd87220fc    3.0
0x00566a2376ed7eac8e5822ba05b3d13293ba7724    3.0
0x005b8746ecf646f0f7481acce3280fc6af2b1ea5    3.0
0x00612bbe504d87cfd25f06feb83fa31f7d4081f6    3.0
0x0064aaa4be87e70b406bfce7ee18bfb00a4a8aa0    3.0
0x0068ccb32a02c28a644985948229c52a62fd6bb3    3.0
0x007dc1a38ef91bc6a55d0f516324beebeaf2f2ea    3.0
0x007f227e41d5b4b16a0afdd09a2c23c80822d99e    3.0
0x0082749dbb009265e80b4fc0bb8eee365429586b    3.0


In [7]:
eth = (
    transactions
    .query('input == "0x"')
    .rename(columns={'block_timestamp': 'timestamp'})
    [['timestamp', 'to_address', 'from_address', 'value']]
)
eth.head()

,timestamp,to_address,from_address,value
0,2022-12-09 00:04:23 UTC,0x3421ca98dc9689e82da4e5cb23f6ad47f8707aa1,0x368f27605a0dc5a8cfd04eec4c6ed80a29697e61,62175500000000000
1,2022-12-09 00:08:11 UTC,0x33fe0557cfe59afdc57b45062ec61acf4f2f3971,0x71660c4005ba85c37ccec55d0c4493e66fe775d3,1159095000000000
2,2022-12-09 00:09:35 UTC,0x0a5df7e60428fca0fea41702edf54408f5161294,0x562fead53d265fc0b28bf6a3ec39bc4b50b5a94d,156141000000000000
3,2022-12-08 23:42:11 UTC,0xf89d7b9c864f589bbf53a82105107622b35eaa40,0xb97bf5f5afc88e34393b6bef38da605999f42f82,4599618625205563000
4,2022-12-08 23:42:11 UTC,0xf6281d13c616900767d70cafdd6b2bd63dd948a5,0xddfabcdc4d8ffc6d5beaf154f18b778f892a0740,321000000000000000


In [8]:
# Example
eth[
    (eth.to_address == '0x0022ae67605e018894cd7ce2893792ccad9c1a00') 
    | (eth.from_address == '0x0022ae67605e018894cd7ce2893792ccad9c1a00')
]

,timestamp,to_address,from_address,value
32344,2022-12-08 15:24:11 UTC,0x0022ae67605e018894cd7ce2893792ccad9c1a00,0x4976a4a02f38326660d17bf34b431dc6e2eb2327,23566340000000000
40413,2022-12-08 15:35:35 UTC,0x0022ae67605e018894cd7ce2893792ccad9c1a00,0x28c6c06298d514db089934071355e5743bf21d60,105777090000000000
189648,2022-12-08 17:46:47 UTC,0xf89d7b9c864f589bbf53a82105107622b35eaa40,0x0022ae67605e018894cd7ce2893792ccad9c1a00,101343430000000000


## USDC Transfers

In [9]:
USDC_CONTRACT_ADDRESS = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
TRANSFER_TOPIC0 = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'

with open("abis/usdc.abi.json") as f:
    USDC_ABI = json.load(f)

In [21]:
usdc_transactions = transactions[transactions.to_address == USDC_CONTRACT_ADDRESS]
usdc_transactions[['transaction_hash', 'block_timestamp', 'from_address', 'to_address', 'value']].sample(10)

,transaction_hash,block_timestamp,from_address,to_address,value
58661,0xad65d60d20283f8b7440b146ba528d9c53fc595474b7...,2022-12-08 14:16:35 UTC,0x2ccc7bf2b86bb544b31c5feae50a14d78c0835e3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
15776,0x5a3fc0aaf70d6a2f07ecb2240cfce9c1356d906a16b7...,2022-12-08 23:39:11 UTC,0x97b9d2102a9a65a26e1ee82d59e42d1b73b68689,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
46654,0x1b272ee13df2c3051857980718ff2f7e022899853d35...,2022-12-08 16:34:59 UTC,0x57bfe30f2f40f4219fd7f48987edd487756a22b0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
213777,0x767836aecb58f3e6866f1ea205e79814536a551f063e...,2022-12-08 17:32:59 UTC,0xc1e1ba972da2df8e5015f3b79059b4a2814b0345,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
170271,0xe946117bd496d85117941b914f7c5f7600f849f3dfc1...,2022-12-08 20:50:11 UTC,0x020da8ea73f2ca62ffd76d78498437af39c7395e,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
192545,0x20f5cb7bcde50c2a343746397038b5de1778357af9e5...,2022-12-08 17:00:11 UTC,0x912fd21d7a69678227fe6d08c64222db41477ba0,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
216454,0xe63f9e58aab8a2d78e005d4085284624b90d1fe485e2...,2022-12-08 16:45:23 UTC,0xd42f958e1c3e2a10e5d66343c4c9a57726e5b4b6,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
135785,0xb4975edb5f7703dbb80dc088e9b606b22ad3591f3722...,2022-12-08 07:15:47 UTC,0xd4bddf5e3d0435d7a6214a0b949c7bb58621f37c,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
11760,0xfacbb317da877984582bc52a9ce863a96425c45b0d83...,2022-12-08 22:32:23 UTC,0xb8dc6ce980b4b3d9d3ff8bfc023fa59587c2fc6e,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0
101538,0x8cb14e7c1dde4f81ac99fb6590af86278313bb98990f...,2022-12-08 11:43:47 UTC,0x3376ebc8dce3453a045a145ab7b1e728b2ed581e,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0


In [11]:
usdc_logs = logs[logs.address == USDC_CONTRACT_ADDRESS].reset_index(drop=True)

usdc_logs['topic0'] = usdc_logs.topics.map(lambda x: x[0])

usdc_logs.head()


,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,topic0
0,137,0x3badabb5347f609b62e045bee7f1e5bc3c51282d1ace...,68,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-08 23:34:47 UTC,16143306,0x90f199a40cca479bd70efa85384bf059bd511a83c2fb...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
1,407,0xe8629fb0a60bc00f6e4773ca90073b63901375cd383e...,143,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:13:23 UTC,16143497,0xd6efa95fcd2ff89afc39bdecb652bec07aa42f644987...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
2,133,0xed4f486554751d3b27657a424bfb65541d4c579e21ce...,45,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:07:23 UTC,16143467,0x01981d3d143b7c75dc44a214e7164147bf3b5a0542a7...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
3,171,0x851252c5ab0e0a752f0fbe8d99ed0369894b09b3c53c...,75,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:07:47 UTC,16143469,0xd75756e7c41d35890b5b111afec801bd9af69e4a727a...,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...
4,271,0xf2cb8d3cc925a0fa9e62c2206b031518c59296a80c06...,89,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:07:59 UTC,16143470,0x93bf128edac58b36192a2223e8120960a74b25eed101...,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...


In [12]:
usdc_transfers = usdc_logs[usdc_logs.topic0 == TRANSFER_TOPIC0]
usdc_transfers.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,topic0
0,137,0x3badabb5347f609b62e045bee7f1e5bc3c51282d1ace...,68,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-08 23:34:47 UTC,16143306,0x90f199a40cca479bd70efa85384bf059bd511a83c2fb...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
1,407,0xe8629fb0a60bc00f6e4773ca90073b63901375cd383e...,143,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:13:23 UTC,16143497,0xd6efa95fcd2ff89afc39bdecb652bec07aa42f644987...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
2,133,0xed4f486554751d3b27657a424bfb65541d4c579e21ce...,45,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:07:23 UTC,16143467,0x01981d3d143b7c75dc44a214e7164147bf3b5a0542a7...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
5,5,0x72ccfad6268db4219c0a5839aeea3e0656ccb94c65f6...,1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:08:59 UTC,16143475,0x48439885aaf5f89691ece1dfb756b058598cac211a68...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
6,55,0xfa60227974b83bb9727785704e73e4e2ccbd46d0221a...,22,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x00000000000000000000000000000000000000000000...,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:09:11 UTC,16143476,0xd4b0b4eb4e8ba6e780f82b4bfd475f8f13348ed92a7c...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...


In [13]:
usdc_transfers.transaction_hash.iloc[0]

'0x3badabb5347f609b62e045bee7f1e5bc3c51282d1ace55bc38e816d2f1911a10'

In [14]:
# usdc = web3.eth.contract(address=web3.toChecksumAddress(USDC_CONTRACT_ADDRESS), abi=USDC_ABI)
# abi.decode(['address', 'address', 'uint256'], bytes.fromhex(usdc_transfers.data.iloc[0].replace('0x', '')))
topic_map = eth_event.get_topic_map(USDC_ABI)
eth_event.decode_log(
    {
        'address': USDC_CONTRACT_ADDRESS,
        'topics': usdc_transfers.topics.iloc[0], 
        'data': usdc_transfers.data.iloc[0], 
    },
    topic_map
)

{'name': 'Transfer',
 'data': [{'name': 'from',
   'type': 'address',
   'value': '0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc',
   'decoded': True},
  {'name': 'to',
   'type': 'address',
   'value': '0xf0e66ec47608b99c6378e28fe395590d989b283e',
   'decoded': True},
  {'name': 'value', 'type': 'uint256', 'value': 186575262, 'decoded': True}],
 'decoded': True,
 'address': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'}

## BAYC Transfers

In [15]:
BAYC_CONTRACT_ADDRESS = '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d'
TRANSFER_TOPIC0 = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'

with open("abis/bayc.abi.json") as f:
    BAYC_ABI = json.load(f)

In [16]:
bayc_logs = logs[logs.address == BAYC_CONTRACT_ADDRESS].reset_index(drop=True)
bayc_logs['topic0'] = bayc_logs.topics.map(lambda x: x[0])

bayc_logs.head()


,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,topic0
0,305,0xde212f337f16be84ca774e675b7f6c38988632955003...,111,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:25:59 UTC,16143560,0x5e9ee9be08cef7c84e6ad0434843e6d8b36414088732...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
1,234,0x705b708d450a6e2fcd30c068271a98c995e3d85c78c1...,41,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:28:23 UTC,16143572,0x29d91c055ab84cc743f4f86b52f57c7e5fcc6f252bd7...,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...
2,63,0xf193c938c1acf70f3a112531fd1da989948c44869362...,11,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:26:59 UTC,16143565,0xddfd0393d0593ab43574368abb136e37f8a7b0f35ee5...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
3,128,0xa1b0e9b51617c40af15ea6ff222ac9a69689352b34bd...,68,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,2022-12-09 00:28:11 UTC,16143571,0xb558fb19c0e54bddab71bd8b383bdfe471c641699166...,0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...
4,129,0xa1b0e9b51617c40af15ea6ff222ac9a69689352b34bd...,68,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:28:11 UTC,16143571,0xb558fb19c0e54bddab71bd8b383bdfe471c641699166...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...


In [17]:
bayc_transfers = bayc_logs[bayc_logs.topic0 == TRANSFER_TOPIC0]
bayc_transfers.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,topic0
0,305,0xde212f337f16be84ca774e675b7f6c38988632955003...,111,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:25:59 UTC,16143560,0x5e9ee9be08cef7c84e6ad0434843e6d8b36414088732...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
2,63,0xf193c938c1acf70f3a112531fd1da989948c44869362...,11,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:26:59 UTC,16143565,0xddfd0393d0593ab43574368abb136e37f8a7b0f35ee5...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
4,129,0xa1b0e9b51617c40af15ea6ff222ac9a69689352b34bd...,68,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:28:11 UTC,16143571,0xb558fb19c0e54bddab71bd8b383bdfe471c641699166...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
6,32,0x8c63012d20a0d3d74555437e86af9feb98bc7c54f1dc...,13,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:23:59 UTC,16143550,0x8f4c955e455fb1962ced8778b89b104f7fa124b17982...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...
13,194,0xc3b5fc39fcd334075e4b933e23b0c25a4744d68504c2...,78,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,0x,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,2022-12-09 00:21:59 UTC,16143540,0xa77b6944590e2734aa678df91b2e88fa13f0c94cd6cc...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...


In [18]:
topic_map = eth_event.get_topic_map(BAYC_ABI)
eth_event.decode_log(
    {
        'address': BAYC_CONTRACT_ADDRESS,
        'topics': bayc_transfers.topics.iloc[0], 
        'data': bayc_transfers.data.iloc[0], 
    },
    topic_map
)

{'name': 'Transfer',
 'data': [{'name': 'from',
   'type': 'address',
   'value': '0x8ae57a027c63fca8070d1bf38622321de8004c67',
   'decoded': True},
  {'name': 'to',
   'type': 'address',
   'value': '0xf3e4fdd7c3eed209040f73f6ce073dfdb5256069',
   'decoded': True},
  {'name': 'tokenId', 'type': 'uint256', 'value': 8114, 'decoded': True}],
 'decoded': True,
 'address': '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D'}